<a href="https://colab.research.google.com/github/aydinhv/customer-churn-analysis/blob/main/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas sqlalchemy matplotlib seaborn scikit-learn joblib


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install faker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.7 MB/s eta 0:00:00


In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from faker import Faker
import random

# Initialize Faker
fake = Faker()

# Set seed for reproducibility
np.random.seed(42)

# Define parameters
num_customers = 1000

# Generate synthetic data
data = {
    'customer_id': range(1, num_customers + 1),
    'gender': np.random.choice(['Male', 'Female'], num_customers),
    'senior_citizen': np.random.choice([0, 1], num_customers),
    'partner': np.random.choice(['Yes', 'No'], num_customers),
    'dependents': np.random.choice(['Yes', 'No'], num_customers),
    'tenure': np.random.randint(1, 73, num_customers),
    'phone_service': np.random.choice(['Yes', 'No'], num_customers),
    'multiple_lines': np.random.choice(['Yes', 'No', 'No phone service'], num_customers),
    'internet_service': np.random.choice(['DSL', 'Fiber optic', 'No'], num_customers),
    'online_security': np.random.choice(['Yes', 'No', 'No internet service'], num_customers),
    'online_backup': np.random.choice(['Yes', 'No', 'No internet service'], num_customers),
    'device_protection': np.random.choice(['Yes', 'No', 'No internet service'], num_customers),
    'tech_support': np.random.choice(['Yes', 'No', 'No internet service'], num_customers),
    'streaming_tv': np.random.choice(['Yes', 'No', 'No internet service'], num_customers),
    'streaming_movies': np.random.choice(['Yes', 'No', 'No internet service'], num_customers),
    'contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], num_customers),
    'paperless_billing': np.random.choice(['Yes', 'No'], num_customers),
    'payment_method': np.random.choice(['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)'], num_customers),
    'monthly_charges': np.round(np.random.uniform(18.25, 118.75, num_customers), 2),
    'total_charges': lambda x: np.round(x['monthly_charges'] * x['tenure'] * np.random.uniform(0.8, 1.2), 2),
    'churn': np.random.choice(['Yes', 'No'], num_customers)
}

# Create DataFrame
df = pd.DataFrame(data)
df['total_charges'] = df.apply(data['total_charges'], axis=1)

# Save DataFrame to CSV for verification
csv_path = '/content/drive/My Drive/customer_data.csv'
df.to_csv(csv_path, index=False)

# Connect to the SQLite database
database_path = '/content/drive/My Drive/customer_data.db'
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Create the table
create_table_query = """
CREATE TABLE IF NOT EXISTS customer_data (
    customer_id INTEGER PRIMARY KEY,
    gender TEXT,
    senior_citizen INTEGER,
    partner TEXT,
    dependents TEXT,
    tenure INTEGER,
    phone_service TEXT,
    multiple_lines TEXT,
    internet_service TEXT,
    online_security TEXT,
    online_backup TEXT,
    device_protection TEXT,
    tech_support TEXT,
    streaming_tv TEXT,
    streaming_movies TEXT,
    contract TEXT,
    paperless_billing TEXT,
    payment_method TEXT,
    monthly_charges REAL,
    total_charges REAL,
    churn TEXT
);
"""

# Convert DataFrame to list of tuples
data_tuples = list(df.itertuples(index=False, name=None))

# Insert data query
insert_data_query = """
INSERT INTO customer_data (customer_id, gender, senior_citizen, partner, dependents, tenure, phone_service, multiple_lines, internet_service, online_security, online_backup, device_protection, tech_support, streaming_tv, streaming_movies, contract, paperless_billing, payment_method, monthly_charges, total_charges, churn)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
"""

# Execute the queries
cursor.execute(create_table_query)
cursor.executemany(insert_data_query, data_tuples)
conn.commit()

# Verify the table content
cursor.execute("SELECT * FROM customer_data LIMIT 5;")
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the cursor and connection
cursor.close()
conn.close()


(1, 'Male', 1, 'Yes', 'No', 7, 'No', 'Yes', 'No', 'Yes', 'Yes', 'No internet service', 'Yes', 'Yes', 'No internet service', 'Month-to-month', 'Yes', 'Bank transfer (automatic)', 68.82, 449.35, 'No')
(2, 'Female', 0, 'No', 'No', 35, 'No', 'No phone service', 'Fiber optic', 'No internet service', 'Yes', 'No', 'No', 'No', 'No', 'Two year', 'No', 'Mailed check', 64.96, 2386.76, 'Yes')
(3, 'Male', 0, 'No', 'No', 55, 'Yes', 'No phone service', 'No', 'Yes', 'Yes', 'No internet service', 'No', 'Yes', 'Yes', 'Month-to-month', 'Yes', 'Credit card (automatic)', 118.5, 7366.36, 'No')
(4, 'Male', 0, 'No', 'No', 14, 'No', 'No phone service', 'Fiber optic', 'Yes', 'No', 'Yes', 'No', 'No internet service', 'No', 'Two year', 'No', 'Mailed check', 67.85, 1115.86, 'No')
(5, 'Male', 0, 'No', 'Yes', 10, 'No', 'No phone service', 'No', 'Yes', 'No internet service', 'No internet service', 'Yes', 'No', 'Yes', 'Month-to-month', 'No', 'Mailed check', 36.18, 405.67, 'No')


In [ ]:
# Reconnect to the database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Example SQL Query: Get average monthly charges by churn status
query = """
SELECT churn, AVG(monthly_charges) as avg_monthly_charges
FROM customer_data
GROUP BY churn;
"""

data = pd.read_sql(query, conn)
print(data)

# Close the cursor and connection
cursor.close()
conn.close()


  churn  avg_monthly_charges
0    No            70.536365
1   Yes            68.978446


In [ ]:
# Save SQL query to a file
sql_query = """
SELECT churn, AVG(monthly_charges) as avg_monthly_charges
FROM customer_data
GROUP BY churn;
"""

with open('/content/drive/My Drive/customer_query.sql', 'w') as file:
    file.write(sql_query)


In [ ]:
# Reconnect to the database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Churn Rate by Contract Type
query_churn_contract = """
SELECT contract, COUNT(*) as total_customers, SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) as churned_customers
FROM customer_data
GROUP BY contract;
"""
df_churn_contract = pd.read_sql(query_churn_contract, conn)
print("Churn Rate by Contract Type")
print(df_churn_contract)

# Average Monthly Charges by Internet Service Type
query_avg_charges_internet = """
SELECT internet_service, AVG(monthly_charges) as avg_monthly_charges
FROM customer_data
GROUP BY internet_service;
"""
df_avg_charges_internet = pd.read_sql(query_avg_charges_internet, conn)
print("Average Monthly Charges by Internet Service Type")
print(df_avg_charges_internet)

# Distribution of Tenure
query_tenure_distribution = """
SELECT tenure, COUNT(*) as customer_count
FROM customer_data
GROUP BY tenure;
"""
df_tenure_distribution = pd.read_sql(query_tenure_distribution, conn)
print("Distribution of Tenure")
print(df_tenure_distribution)

# Close the cursor and connection
cursor.close()
conn.close()


Churn Rate by Contract Type
         contract  total_customers  churned_customers
0  Month-to-month              341                168
1        One year              342                171
2        Two year              317                163
Average Monthly Charges by Internet Service Type
  internet_service  avg_monthly_charges
0              DSL            69.050375
1      Fiber optic            70.399485
2               No            69.789543
Distribution of Tenure
    tenure  customer_count
0        1              10
1        2               8
2        3              14
3        4              13
4        5               9
..     ...             ...
67      68              18
68      69              13
69      70              13
70      71              10
71      72              14

[72 rows x 2 columns]


In [ ]:
# Reconnect to the database to load data into a DataFrame
conn = sqlite3.connect(database_path)
df = pd.read_sql("SELECT * FROM customer_data", conn)

# Close the connection
conn.close()

# Display the first few rows of the DataFrame
df.head()


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,1,Male,1,Yes,No,7,No,Yes,No,Yes,...,No internet service,Yes,Yes,No internet service,Month-to-month,Yes,Bank transfer (automatic),68.82,449.35,No
1,2,Female,0,No,No,35,No,No phone service,Fiber optic,No internet service,...,No,No,No,No,Two year,No,Mailed check,64.96,2386.76,Yes
2,3,Male,0,No,No,55,Yes,No phone service,No,Yes,...,No internet service,No,Yes,Yes,Month-to-month,Yes,Credit card (automatic),118.50,7366.36,No
3,4,Male,0,No,No,14,No,No phone service,Fiber optic,Yes,...,Yes,No,No internet service,No,Two year,No,Mailed check,67.85,1115.86,No
4,5,Male,0,No,Yes,10,No,No phone service,No,Yes,...,No internet service,Yes,No,Yes,Month-to-month,No,Mailed check,36.18,405.67,No


In [ ]:
!pip install plotly dash


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 19.2 MB/s eta 0:00:00


In [2]:
# Install dependencies
!pip install dash plotly pandas numpy faker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.1 MB/s eta 0:00:00


In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
from faker import Faker

fake = Faker()
np.random.seed(42)
num_customers = 1000
data = {
    'customer_id': range(1, num_customers + 1),
    'gender': np.random.choice(['Male', 'Female'], num_customers),
    'senior_citizen': np.random.choice([0, 1], num_customers),
    'partner': np.random.choice(['Yes', 'No'], num_customers),
    'dependents': np.random.choice(['Yes', 'No'], num_customers),
    'tenure': np.random.randint(1, 73, num_customers),
    'phone_service': np.random.choice(['Yes', 'No'], num_customers),
    'multiple_lines': np.random.choice(['Yes', 'No', 'No phone service'], num_customers),
    'internet_service': np.random.choice(['DSL', 'Fiber optic', 'No'], num_customers),
    'online_security': np.random.choice(['Yes', 'No', 'No internet service'], num_customers),
    'online_backup': np.random.choice(['Yes', 'No', 'No internet service'], num_customers),
    'device_protection': np.random.choice(['Yes', 'No', 'No internet service'], num_customers),
    'tech_support': np.random.choice(['Yes', 'No', 'No internet service'], num_customers),
    'streaming_tv': np.random.choice(['Yes', 'No', 'No internet service'], num_customers),
    'streaming_movies': np.random.choice(['Yes', 'No', 'No internet service'], num_customers),
    'contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], num_customers),
    'paperless_billing': np.random.choice(['Yes', 'No'], num_customers),
    'payment_method': np.random.choice(['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)'], num_customers),
    'monthly_charges': np.round(np.random.uniform(18.25, 118.75, num_customers), 2),
    'total_charges': lambda x: np.round(x['monthly_charges'] * x['tenure'] * np.random.uniform(0.8, 1.2), 2),
    'churn': np.random.choice(['Yes', 'No'], num_customers)
}

df = pd.DataFrame(data)
df['total_charges'] = df.apply(data['total_charges'], axis=1)

# Additional Data Preparation
df_churn_contract = df.groupby('contract').agg(
    total_customers=pd.NamedAgg(column='customer_id', aggfunc='count'),
    churned_customers=pd.NamedAgg(column='churn', aggfunc=lambda x: (x == 'Yes').sum())
).reset_index()

df_avg_charges_internet = df.groupby('internet_service').agg(
    avg_monthly_charges=pd.NamedAgg(column='monthly_charges', aggfunc='mean')
).reset_index()

df_tenure_distribution = df.groupby('tenure').agg(
    customer_count=pd.NamedAgg(column='customer_id', aggfunc='count')
).reset_index()

# Customer Demographics Analysis
fig_gender_distribution = px.pie(
    df,
    names='gender',
    title='Gender Distribution',
    color='gender'
)

fig_senior_citizen_distribution = px.pie(
    df,
    names='senior_citizen',
    title='Senior Citizen Distribution',
    labels={'senior_citizen': 'Senior Citizen'},
    color='senior_citizen'
)

# Service Usage Patterns
fig_multiple_lines_usage = px.bar(
    df,
    x='multiple_lines',
    title='Multiple Lines Usage',
    labels={'multiple_lines': 'Multiple Lines Usage'},
    color='multiple_lines'
)

fig_internet_service_types = px.pie(
    df,
    names='internet_service',
    title='Internet Service Types',
    color='internet_service'
)

# Payment Method Preferences
fig_payment_methods = px.pie(
    df,
    names='payment_method',
    title='Payment Method Preferences',
    color='payment_method'
)

# Churn Rate by Contract Type (Pie Chart)
fig_churn_contract_pie = px.pie(
    df_churn_contract,
    names='contract',
    values='churned_customers',
    title='Churn Rate by Contract Type',
    color='contract'
)

# Average Monthly Charges by Internet Service Type (Line Chart)
fig_avg_charges_internet_line = px.line(
    df_avg_charges_internet,
    x='internet_service',
    y='avg_monthly_charges',
    title='Average Monthly Charges by Internet Service Type',
    markers=True,
    line_shape='spline'
)

# Distribution of Tenure (Box Plot)
fig_tenure_distribution_box = px.box(
    df,
    x='tenure',
    y='monthly_charges',
    color='churn',
    title='Distribution of Tenure vs Monthly Charges',
    points='all'
)

# Churn vs Monthly Charges Scatter Plot (Enhanced)
fig_churn_monthly_enhanced = px.scatter(
    df,
    x='monthly_charges',
    y='total_charges',
    color='churn',
    title='Churn vs Monthly Charges',
    labels={'monthly_charges': 'Monthly Charges', 'total_charges': 'Total Charges'},
    hover_data=['tenure', 'contract', 'payment_method'],
    size='tenure',
    size_max=15
)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("Customer Churn Analysis Dashboard"),

    dcc.Tabs([
        dcc.Tab(label='Churn Rate by Contract Type', children=[
            dcc.Graph(id='churn-contract-graph', figure=fig_churn_contract_pie)
        ]),
        dcc.Tab(label='Avg Monthly Charges by Internet Service Type', children=[
            dcc.Graph(id='avg-charges-internet-graph', figure=fig_avg_charges_internet_line)
        ]),
        dcc.Tab(label='Distribution of Tenure vs Monthly Charges', children=[
            dcc.Graph(id='tenure-distribution-graph', figure=fig_tenure_distribution_box)
        ]),
        dcc.Tab(label='Churn vs Monthly Charges', children=[
            dcc.Graph(id='churn-monthly-graph', figure=fig_churn_monthly_enhanced)
        ]),
        dcc.Tab(label='Customer Demographics', children=[
            dcc.Graph(id='gender-distribution-graph', figure=fig_gender_distribution),
            dcc.Graph(id='senior-citizen-distribution-graph', figure=fig_senior_citizen_distribution)
        ]),
        dcc.Tab(label='Service Usage Patterns', children=[
            dcc.Graph(id='multiple-lines-usage-graph', figure=fig_multiple_lines_usage),
            dcc.Graph(id='internet-service-types-graph', figure=fig_internet_service_types)
        ]),
        dcc.Tab(label='Payment Method Preferences', children=[
            dcc.Graph(id='payment-methods-graph', figure=fig_payment_methods)
        ])
    ])
])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>